In [1]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf

In [2]:
num_epochs = 100 #训练时epoch的数量
total_series_length = 50000 #输入序列的最大长度
truncated_backprop_length = 15 #每次训练RNN时，读取时间序列的长度
state_size = 4 #状态矢量的size
num_classes = 2 #输出类别数量，对于0和1来说就是2
echo_step = 3 #偏移量
batch_size = 5 #batch的大小
num_batches = total_series_length//batch_size//truncated_backprop_length #batch的数量

In [3]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))
    y = y.reshape((batch_size, -1))

    return (x, y)

In [4]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [5]:
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [6]:
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [7]:
current_state = init_state # 目前的状态矢量
states_series = [] # 所有状态矢量放到一个list中
for current_input in inputs_series: # 遍历输入数据
    current_input = tf.reshape(current_input, [batch_size, 1]) # 获取当前输入
    input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # 将输入和状态合，成为新的输入

    next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  #用此输入生成新的状态
    states_series.append(next_state)
    current_state = next_state

In [8]:
logits_series = [tf.matmul(state, W2) + b2 for state in states_series] 
predictions_series = [tf.nn.softmax(logits) for logits in logits_series] #利用softmax预测输出

In [9]:
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())  # 变量初始化

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size)) # 初始状态设置为0

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches): # 按batch训练
            start_idx = batch_idx * truncated_backprop_length # 当前输入数据的起点
            end_idx = start_idx + truncated_backprop_length # 当前输入数据的重点

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            # 输入数据，开始训练
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            # 观察loss的变化
            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
New data, epoch 0
Step 0 Loss 0.93254304
Step 100 Loss 0.70382816
Step 200 Loss 0.7018951
Step 300 Loss 0.6851621
Step 400 Loss 0.69314176
Step 500 Loss 0.6313028
Step 600 Loss 0.05302203
New data, epoch 1
Step 0 Loss 0.15994495
Step 100 Loss 0.010874013
Step 200 Loss 0.006345859
Step 300 Loss 0.004856853
Step 400 Loss 0.0036119751
Step 500 Loss 0.0033813266
Step 600 Loss 0.0029474346
New data, epoch 2
Step 0 Loss 0.13597406
Step 100 Loss 0.0021615117
Step 200 Loss 0.0022674522
Step 300 Loss 0.0015539691
Step 400 Loss 0.0015417513
Step 500 Loss 0.0014509642
Step 600 Loss 0.0014781492
New data, epoch 3
Step 0 Loss 0.19722544
Step 100 Loss 0.0015681108
Step 200 Loss 0.0012928752
Step 300 Loss 0.0011466461
Step 400 Loss 0.0010695847
Step 500 Loss 0.0010120411
Step 600 Loss 0.0008637632
New data, epoch 4
Step 0 Loss 0.14180847
Step 100 Loss 0.02151207
Step 200 Loss 0.009041339
Step 300 Loss 0.006129089
Step 400 Loss 

Step 200 Loss 0.0001296151
Step 300 Loss 0.00013657629
Step 400 Loss 0.00013930342
Step 500 Loss 0.0001106196
Step 600 Loss 0.00011891366
New data, epoch 39
Step 0 Loss 0.14024702
Step 100 Loss 0.00012164598
Step 200 Loss 0.00013608266
Step 300 Loss 0.000104805855
Step 400 Loss 0.00012383825
Step 500 Loss 0.000118985074
Step 600 Loss 0.00012653781
New data, epoch 40
Step 0 Loss 0.17054401
Step 100 Loss 0.000113413895
Step 200 Loss 0.00012550016
Step 300 Loss 0.00012090068
Step 400 Loss 0.00012332755
Step 500 Loss 0.000106101324
Step 600 Loss 0.00011492846
New data, epoch 41
Step 0 Loss 0.13512297
Step 100 Loss 0.000103339364
Step 200 Loss 0.000121468205
Step 300 Loss 0.00012647112
Step 400 Loss 0.00010534685
Step 500 Loss 0.000118092736
Step 600 Loss 0.000103365026
New data, epoch 42
Step 0 Loss 0.16456261
Step 100 Loss 0.00011343803
Step 200 Loss 0.000112447655
Step 300 Loss 0.0001183803
Step 400 Loss 0.000112271126
Step 500 Loss 0.00010158852
Step 600 Loss 0.00011573577
New data, epo